In [1]:
import pandas as pd 
import numpy as np
import seaborn as sb

import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
airdata = pd.read_csv("datasets/datasets/airquality.csv")

/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
#check number of rows
len(airdata)


146

In [20]:
#check Number of Columns
len(airdata.columns)

7

In [21]:
#view sample data
airdata.head()

,Unnamed: 0,Ozone,Solar.R,Wind,Temp,Month,Day
0,1,41.0,190.0,7.4,67,5,1
1,2,36.0,118.0,8.0,72,5,2
2,3,12.0,149.0,12.6,74,5,3
3,4,18.0,313.0,11.5,62,5,4
6,7,23.0,299.0,8.6,65,5,7


In [2]:
#check null in data
airdata.isnull().sum()

Unnamed: 0     0
Ozone         37
Solar.R        7
Wind           0
Temp           0
Month          0
Day            0
dtype: int64

In [6]:
#Get mean of the Ozone and mean ozone by every month
Avgofozone_month = airdata.groupby(['Month'])[["Ozone"]].mean()
Avgofozone = airdata['Ozone'].mean()
Avgofozone_month

In [8]:
# Sample get mean of the Ozone based on Month
airdata.loc[airdata['Month'] == 5].groupby(['Month'])[["Ozone"]].mean()

,Ozone
Month,
5,23.615385


In [28]:
sampl = airdata[['Ozone','Month']]

In [3]:
#function for adding values to the NA in Ozone
#this function calculate the value mean of Ozone for every month,then replace the NA with mean values accorcingly
def month_avg(coll,mont):
    reslt = coll.loc[airdata['Month'] == mont].groupby(['Month'])[["Ozone"]].mean()
    return reslt['Ozone'].round().astype(np.int64)


def Ozone_approx(row):
    Ozone = row['Ozone']
    Month = row['Month']
    
    if pd.isnull(Ozone):
        res = month_avg(airdata[['Ozone','Month']],Month)
        return res.to_frame().reset_index().iloc[0]['Ozone']
    else:
        return Ozone


In [108]:

ne = month_avg(airdata[['Ozone','Month']],5)

# get the value alone from the DataFrame Cell
ne.to_frame().reset_index().iloc[0]['Ozone']

In [4]:
#apply the Ozone_approx on the Ozone column to handle the NA
airdata['Ozone'] = airdata.apply(Ozone_approx, axis=1)

In [5]:
#the other column that has NA is solar.R with only 4% of the data so we can remove those NA. 
#in some cases we have handle then them too
airdata.dropna(inplace=True)
#to check the values
airdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 152
Data columns (total 7 columns):
Unnamed: 0    146 non-null int64
Ozone         146 non-null float64
Solar.R       146 non-null float64
Wind          146 non-null float64
Temp          146 non-null int64
Month         146 non-null int64
Day           146 non-null int64
dtypes: float64(3), int64(4)
memory usage: 9.1 KB


In [12]:
#it gives the stastical view of the Data in the Dataframe
airdata.describe()

,Unnamed: 0,Ozone,Solar.R,Wind,Temp,Month,Day
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,78.363014,40.958904,185.931507,10.003425,78.116438,7.027397,16.123288
std,43.935514,29.867726,90.058422,3.508795,9.224787,1.404157,8.794897
min,1.000000,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000
25%,41.250000,21.000000,115.750000,7.400000,73.000000,6.000000,9.000000
50%,77.500000,29.000000,205.000000,9.700000,79.000000,7.000000,16.000000
75%,116.750000,59.000000,258.750000,11.500000,84.000000,8.000000,23.750000
max,153.000000,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


In [14]:
corrmap = sb.heatmap(airdata.corr())

In [15]:
corrmap

In [16]:
airdata.apply(lambda s: airdata.corrwith(s))

,Unnamed: 0,Ozone,Solar.R,Wind,Temp,Month,Day
Unnamed: 0,1.000000,0.192904,-0.104682,-0.153270,0.345917,0.979783,0.193140
Ozone,0.192904,1.000000,0.298007,-0.523111,0.649315,0.201634,-0.013029
Solar.R,-0.104682,0.298007,1.000000,-0.056792,0.275840,-0.075301,-0.150275
Wind,-0.153270,-0.523111,-0.056792,1.000000,-0.453847,-0.162253,0.025307
Temp,0.345917,0.649315,0.275840,-0.453847,1.000000,0.380970,-0.131936
Month,0.979783,0.201634,-0.075301,-0.162253,0.380970,1.000000,-0.006977
Day,0.193140,-0.013029,-0.150275,0.025307,-0.131936,-0.006977,1.000000


In [30]:
airdata.columns

Index([u'Unnamed: 0', u'Ozone', u'Solar.R', u'Wind', u'Temp', u'Month', u'Day',
       u'Result'],
      dtype='object')

In [7]:
#Add windy climate or not
def find_windy(row):
    
    Wind = row['Wind']
    
    if Wind > 11.5 :
        return 1
    else:
        return 0

In [8]:
#To Drop a Column
#airdata.drop(['Result'],inplace=True,axis=1)

#apply find windy logic to new column
airdata['Result'] = airdata.apply(find_windy, axis=1)

In [22]:
#final dataset
airdata_fnl = airdata[['Ozone','Solar.R', 'Temp', 'Month', 'Day','Result']]

In [23]:
#spliting Iependent vairable and independent variables
X = airdata_fnl.ix[:,(0,1,2,3,4)].values # independent variables
y = airdata_fnl.ix[:,5].values # Dependent variables

In [24]:
#creating training and test Datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=25)

In [25]:
#creating logisting regfession model from training Data
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
#apply the model on the test data
y_pred = LogReg.predict(X_test)

In [27]:
#check confusion matris determine the model efficiency  
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

array([[23,  0],
       [ 7,  0]])

In [36]:
#score
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.77      1.00      0.87        23
          1       0.00      0.00      0.00         7

avg / total       0.59      0.77      0.67        30



In [30]:
#view results side by side
pd.DataFrame(y_test, y_pred)

,0
0,0
0,0
0,1
0,0
0,0
0,0
0,0
0,0
0,0
0,1
